## Length Selector

In [1]:
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from pprint import pprint

import tiktoken

# tiktoken.count("")
# global ENCODING_CONSTRUCTORS
# pprint(ENCODING_CONSTRUCTORS)

examples = [
    {"input": "高兴", "output": "伤心"},
    {"input": "高", "output": "矮"},
    {"input": "积极", "output": "消极"},
    {"input": "晴天", "output": "阴天"},
    {"input": "前面", "output": "后面"},
]
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}"
)

def get_length(str: str) -> int:
    return len(tiktoken.get_encoding('cl100k_base').encode(str))

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=64,
    get_text_length=get_length
)

dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出所有输入的反义词",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
    example_separator="\n"
)

### 用户输入内容长度小于25token

In [2]:
pprint(dynamic_prompt.format(adjective="开心"))

('给出所有输入的反义词\n'
 'Input: 高兴\n'
 'Output: 伤心\n'
 'Input: 高\n'
 'Output: 矮\n'
 'Input: 积极\n'
 'Output: 消极\n'
 'Input: 晴天\n'
 'Output: 阴天\n'
 'Input: 前面\n'
 'Output: 后面\n'
 'Input: 开心\n'
 'Output:')


### 长度大于32 token

In [3]:
input = "很高兴听到你感到开心！😊 有什么特别的原因让你心情这么好吗？"
pprint("user input length: {}".format(get_length(input)))
pprint(dynamic_prompt.format(adjective=input))
# pprint(len("很高兴听到你感到开心！😊 有什么特别的原因让你心情这么好吗？高兴高兴高兴高兴高兴高兴高兴高兴高兴高兴高兴高兴高兴高兴高兴高兴高兴，你好"))

'user input length: 36'
('给出所有输入的反义词\n'
 'Input: 高兴\n'
 'Output: 伤心\n'
 'Input: 高\n'
 'Output: 矮\n'
 'Input: 很高兴听到你感到开心！😊 有什么特别的原因让你心情这么好吗？\n'
 'Output:')
